In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd gdrive/My\ Drive/Information\ retrieval/Project\ 1

/content/gdrive/My Drive/Information retrieval/Project 1


In [ ]:
# All Articles
# !wget "https://dumps.wikimedia.org/fawiki/20201120/fawiki-20201120-pages-articles-multistream.xml.bz2"
# !bzip2 -d fawiki-20201120-pages-articles-multistream.xml.bz2

In [ ]:
# Abstract
# !wget "https://dumps.wikimedia.org/fawiki/20201101/fawiki-20201101-abstract.xml.gz"
# !gunzip fawiki-20201101-abstract.xml.gz

In [ ]:
!ls -lh

total 6.7G
-rw------- 1 root root 1.4K Dec  8 16:06  charsSet.txt
-rw------- 1 root root  459 Dec  8 15:10  chars.txt
-rw------- 1 root root 280K Dec 16 08:02  DesignSeq2SeqModel.ipynb
-rw------- 1 root root 709M Nov  4 01:16  fawiki-20201101-abstract.xml
-rw------- 1 root root 5.8G Nov 21 16:25  fawiki-20201120-pages-articles-multistream.xml
drwx------ 2 root root 4.0K Dec 16 07:10  savedmodels
-rw------- 1 root root  43K Dec 25 20:36  SpellCorrectionPreProcess.ipynp
drwx------ 2 root root 4.0K Dec 16 07:28  training_checkpoints
-rw------- 1 root root 4.8M Dec 12 14:04 'VocabularyAbstract_v1.0-36char-afterTokenize&RemoveUnderlineSplitBy|.txt'
-rw------- 1 root root 4.8M Dec 21 15:37 'VocabularyAbstract_v1.0-36char-Tokenize&RemoveUnderline2Words3WordsSplitByN-ReadyToBuildDataSet.txt'
-rw------- 1 root root  22M Dec  8 19:23  Vocabulary.txt
-rw------- 1 root root  23M Dec  9 09:17  Vocabulary_v1.2.txt
-rw------- 1 root root  19M Dec  9 12:20  Vocabulary_v2.1.txt
-rw------- 1 root root  

In [ ]:
!head -n30 fawiki-20201101-abstract.xml

In [ ]:
import pandas as pd
import re
import string

In [ ]:
from lxml import etree

class Wikipedia:
    def __init__(self, fh, tag):
        """
        Initialize 'iterparse' to only generate 'end' events on tag '<entity>'

        :param fh: File Handle from the XML File to parse
        :param tag: The tag to process
        """
        # Prepend the default Namespace {*} to get anything.
        self.context = etree.iterparse(fh, events=("end",), tag=['{*}' + tag])

    def _parse(self):
        """
        Parse the XML File for all '<tag>...</tag>' Elements
        Clear/Delete the Element Tree after processing

        :return: Yield the current 'Event, Element Tree'
        """
        for event, elem in self.context:
            yield event, elem

            elem.clear()
            while elem.getprevious() is not None:
                del elem.getparent()[0]

    def __iter__(self):
        """
        Iterate all '<tag>...</tag>' Element Trees yielded from self._parse()

        :return: Dict var 'entity' {tag1, value, tag2, value, ... ,tagn, value}}
        """
        for event, elem in self._parse():
            entity = {}

            # Assign the 'elem.namespace' to the 'xpath'
            entity['revision'] = elem.xpath('./xmlns:revision/xmlns:text/text( )', 
                                   namespaces={'xmlns':etree.QName(elem).namespace})

            yield entity

### Normalize and save and tokenize


In [ ]:
p = 0
with open('fawiki-20201120-pages-articles-multistream.xml', 'rb') as in_xml:
    chars = []
    for record in Wikipedia(in_xml, tag='page'):
        try:
            chars.extend(list(set(list(str(record['revision'][0])))))
        except Exception as ex:
            pass
        # break
        if p%100000==0:
            chars = list(set(chars))
            print(p//100000, 'Len', len(chars), end=' ')
        p += 1
print(p)

0 Len 108 1 Len 5736 2 Len 6817 3 Len 7513 4 Len 7744 5 Len 8184 6 Len 8825 7 Len 8964 8 Len 9142 9 Len 9191 10 Len 9245 11 Len 9390 12 Len 9439 13 Len 9554 14 Len 9573 15 Len 9625 16 Len 9893 17 Len 10031 18 Len 10135 19 Len 10726 20 Len 10808 21 Len 10882 22 Len 11103 23 Len 11300 24 Len 11476 25 Len 11680 26 Len 12188 27 Len 12309 28 Len 12339 29 Len 12786 30 Len 13082 31 Len 13207 32 Len 13409 33 Len 13749 3358604


In [ ]:
# Abstract
import xml.etree.ElementTree as ET
from time import time
root = ET.parse('fawiki-20201101-abstract.xml').getroot()
chars = []
for i, item in enumerate(root.findall('./doc')):
    if i%100000==0:
        print(i//100000, end=' ')
    if len(item)>2: 
        chars.extend(list(set(list(str(item[2].text))))) # item[2] is document body
print("All Done.")

0 1 2 3 4 5 6 7 All Done.


In [ ]:
whitelist = list('\u200cآ-ـضصثقفغعهخحجچگکمنتالبیسشظطزرذدپوژ\u200d ٓ')
stopchars = []
for c in chars:
    if not (c in whitelist):
        stopchars.append(c)
stopchars = list(set(stopchars))
len(stopchars)

13967

In [ ]:
class Normalizer:
    def __init__(self):
        import re
        self.whitelist = list('\u200cآ-ـضصثقفغعهخحجچگکمنتالبیسشظطزرذدپوژ\u200d')
        maketrans = lambda A, B: dict((ord(a), b) for a, b in zip(A, B))
        compile_patterns = lambda patterns: [(re.compile(pattern), repl) for pattern, repl in patterns]
        translation_src = ' ىكيئ“”ةأإؤۄ'   # str(''.join(chars))
        translation_dst = ' یکیی""هااوو'   # str(''.join(newchars))
        translation_src += '?!'
        translation_dst += '؟!'
        self.translations = maketrans(translation_src, translation_dst)
        
        self.character_refinement_patterns = []
        self.character_refinement_patterns.extend([
            # (r'[ـ\r^A-Za-z0-9۰-۹\<\>\-\=\+\\\/\{\}\|\[\]:;"&%٪#$\*\'_]', ''),  # remove keshide, English words, numbers, carriage returns
            # ('"([^\n"]+)"', r'\1'),  # replace quotation with gyoome
            ('[\u064B\u064C\u064D\u064E\u064F\u0650\u0651\u0652\u0654\u0621]', ''),  # remove FATHATAN, DAMMATAN, KASRATAN, FATHA, DAMMA, KASRA, SHADDA, SUKUN
            (r'[\s \=\|\(\)\[\]\{\}\:\+\?\؟\!\!\.\,\;\'\"\@\#\$\%\^\&\*\`\~\<\>\/\\\٬\٫\﷼\٪\×\،\*\=\+\÷\\\\\:۱۲۳۴۵۶۷۸۹۰ ٠١٢٣٤٥٦٧٨٩\«\»]+', ' '),  #
            (r'\n+', '\n'),  # remove extra newlines
            (r'\n.\n.+', '\n'),  # remove extra newlines
            (r'\n+', '\n'),  # remove extra newlines
        ])
        self.character_refinement_patterns = compile_patterns(self.character_refinement_patterns)
    def normalizeContent(self, text):
        text = text.translate(self.translations)
        for pattern, repl in self.character_refinement_patterns:
            text = pattern.sub(repl, text)
        return text
    def normalizeTokens(self, token, pat):  # if token is acceptable (persian and lengh>1) return Normalized token, else return None
        # print("NormalizingTokens Started...")
        token = re.sub(pat, '', str(token))
        c = 1
        out = False
        if len(token)>1:
            for i in range(len(token)-1):
                if token[i] == token[i+1]:
                    c+=1
                    if c>2:
                        out = True
                        break
                else :
                    c = 1
            if len(token)>1 and self.ispersian(token) and not out:
                return token
        return None

    def tokenize(self, content):
        paterns = '[?؟\., !!،»«\(\)\[\]\-\s\n::A-Za-z%\\\#\$\&\^\*\+\=\/\<\>\@\{\}\~\'\"0123456789۰۱۲۳۴۵۶۷۸۹٠١٢٣٤٥٦٧٨٩ \s/\|]+'
        content = re.split(paterns, content)
        # print(content)
        return list(set(content))

    def getFreshTokens(self, article, pat):
        newArt = []
        for token in article:
            token = self.normalizeTokens(token, pat)
            if token != None:
                newArt.append(token)
        return newArt

    def ispersian(self, token):
        token = list(token)
        i = 0
        for c in token:
            if c in whitelist:
                i += 1
        if i/len(token) > 0.7:
            return True
        return False

    def getStopChars(self, articles):
        print("Finding Char Set...")
        chars = []
        for i, token in enumerate(l):
            tchars = list(token)
            chars.extend(tchars)
            if i%200000==0:
                chars = list(set(chars))
                print(len(chars), end=" ")
        chars = list(set(chars))
        print(len(chars), end=" ")
        stopchar = []
        for c in chars:
            if not (c in whitelist):
                stopchar.append(c)
        return stopchar

norm = Normalizer()


In [ ]:
stopchars = "".join(stopchars)
stopchars = re.sub(r'[\s \=\|\(\)\[\]\{\}\:\+\?\؟\!\!\.\,\;\'\"\@\#\$\%\^\&\*\`\~\<\>\/\\\٬\٫\﷼\٪\×\،\*\=\+\÷\\\\\:۱۲۳۴۵۶۷۸۹۰ ٠١٢٣٤٥٦٧٨٩\«\»_]+', '', stopchars)
pat = r"[" + stopchars + ' ٍ ٌ ْ ً ُ ِ َ ّ \:\\«\»\ ٔء \ ٬\؍٫ \ ٰ ٓ              \u202e\u202c\u200e \u202d\u0600\u200f\u0602 ـ\ـ' + "]+"

In [ ]:
pat.find('_')

-1

In [ ]:
re.sub(pat, '', '１Ǡ险闻調ﺷ螂sadfشسیبشیب🐍س_بسشــــیب 繕流焼絡牧維')

'شسیبشیبس_بسشیب'

In [ ]:
norm.getFreshTokens(norm.tokenize(norm.normalizeContent("１Ǡ险闻شیب調شسیبﺷ螂شسیبشیب🐍سبسشیب 繕流焼絡牧維")), pat)

['شیبشسیبشسیبشیبسبسشیب']

In [ ]:
# Articles
p = 0
with open('fawiki-20201120-pages-articles-multistream.xml', 'rb') as in_xml:
    Tokens = []
    articles = []
    for record in Wikipedia(in_xml, tag='page'):
        # print(norm.normalizer(record['revision'][0]))
        try:
            article = norm.tokenize(norm.normalizeContent(record['revision'][0]))
            article = norm.getFreshTokens(article, pat)
            articles.append(article)
            Tokens.extend(article)
        except Exception as ex:
            pass
        # break
        if p%100000==0:
            Tokens = list(set(Tokens))
            print(p//100000, len(Tokens), end=' ')
            # with open("fawiki-articles-normalized.txt", 'a+') as nf:
            #     nf.writelines(l)
            # l = []
        p += 1
Tokens = list(set(Tokens))
print(p//100000, len(Tokens), end=' ')

0 58 1 412644 2 521851 3 580828 4 606206 5 637657 6 683590 7 706455 8 721007 9 724757 10 732584 11 759837 12 767698 13 787518 14 791046 15 805791 16 814794 17 838720 18 862006 19 889675 20 906355 21 925528 22 964099 23 984898 24 1026406 25 1052223 26 1075883 27 1100776 28 1107859 29 1120711 30 1142157 31 1210634 32 1235463 33 1263499 33 1283000 

In [ ]:
# Abstracts
import xml.etree.ElementTree as ET
from time import time
root = ET.parse('fawiki-20201101-abstract.xml').getroot()
Tokens = []
p = 0
for i, item in enumerate(root.findall('./doc')):
    if len(item)>2:
        try:
            article = norm.tokenize(norm.normalizeContent(item[2].text))
            article = norm.getFreshTokens(article, pat)
            Tokens.extend(article)
        except Exception as ex:
            pass 
    if p%100000==0:
        Tokens = list(set(Tokens))
        print(p//100000, len(Tokens), end=' ')
    p += 1
    # TODO 
Tokens = list(set(Tokens))
print(p//100000, len(Tokens), end=' ')

0 0 1 87918 2 126096 3 166273 4 197538 5 244991 6 263097 7 287454 7 303634 

In [ ]:
articles[0]

In [ ]:
TokensNum = {el:0 for el in Tokens}
p = 0
for article in articles:
    for token in article:
        if TokensNum[token]>3:
            continue
        try:
            TokensNum[token] += 1
        except:
            pass
    if p%100000==0:
        print(p//100000, end=' ')
    p += 1

0 0 1 100000 2 200000 3 300000 4 400000 5 500000 6 600000 7 700000 8 800000 9 900000 10 1000000 11 1100000 12 1200000 13 1300000 14 1400000 15 1500000 16 1600000 17 1700000 18 1800000 19 1900000 20 2000000 21 2100000 22 2200000 23 2300000 24 2400000 25 2500000 26 2600000 27 2700000 28 2800000 29 2900000 30 3000000 31 3100000 32 3200000 33 3300000 

In [ ]:
for key, val in zip(list(TokensNum.keys())[:100],list(TokensNum.values())[:100]):
    print(key, val)

In [ ]:
newTokens = []
p = 0
for token, num in TokensNum.items():
    if num>3:
        newTokens.append(token)
    if p%100000==0:
        print(p//100000, end=' ')
    p += 1
print('\n', len(newTokens))

0 1 2 3 4 5 6 7 8 9 10 11 12 
 396788


In [ ]:
newTokens.sort()
newTokens[:10]

['__بی\u200cنگارخانه__',
 '__بی\u200cپیوندبخش__',
 '__تغییرمسیرثابت__',
 '_آبان',
 '_آبرامز',
 '_آذر',
 '_آرژانتین',
 '_آریزونا',
 '_آلاباما',
 '_آلاسکا']

In [ ]:
newTokenss = newTokens.copy()

In [ ]:
newTokenss = [str.replace(token, '_', ' ') for token in newTokenss]
newTokenss = [str.strip(token) for token in newTokenss]
newTokenss = list(set(newTokenss))
# newTokenss.sort()

In [ ]:
newTokenss.sort()
newTokenss[:100]

In [ ]:
with open('Vocabulary_v5.0-36char-afterTokenize&RemoveUnderlineSplitByN.txt', 'w') as f:
    f.write('\n'.join(newTokenss))

In [ ]:
newTokens = []
for token in newTokenss:
    newTokens.append(token)
    token = token.split(' ')
    if len(token)>1:
        for word in token:
            if len(word)>1:
                newTokens.append(word)
        if len(token)==3:
            for i in range(len(token)-1):
                newTokens.append(str(token[i]+' '+token[i+1]))
        if len(token)>3:
            for i in range(len(token)-2):
                newTokens.append(str(token[i]+' '+token[i+1]+' '+token[i+2]))
newTokens = [str.strip(token) for token in newTokens]
newTokens = list(set(newTokens))
# newTokens.sort()
len(newTokens), newTokens[:10]

(1341556,
 ['دیکست',
  'باکتریوفاژ رشته\u200cای',
  'روففورت',
  'قبیح\u200cاللقا',
  'حاشبیه',
  'تشکیک\u200cکردن\u200cهای',
  'نلدوانی',
  'ایمحوتب',
  'تاکاکازو',
  'تله\u200cنولا'])

In [ ]:
with open('Vocabulary_v4.0-36char-Tokenize&RemoveUnderline2Words3WordsSplitByN-ReadyToBuildDataSet.txt', 'w') as f:
    f.write('\n'.join(newTokens))

In [ ]:
len(newTokens)

1341556

### Draft

In [ ]:
!head -n400 fawiki-20201120-pages-articles-multistream.xml

In [ ]:
chars = []
for i, token in enumerate(l):
    t = list(token)
    for c in t:
        if (ord(c)>=1536 and ord(c)<=1610) or (ord(c)>=1646 and ord(c)<=1749) or (ord(c)>=1786 and ord(c)<=1790) or (ord(c)>=1872 and ord(c)<=1957) \
             or (ord(c)>=2208 and ord(c)<=2237) or (ord(c)>=8192 and ord(c)<=8209):
            chars.append(c)
    if i%100000==0:
        chars = list(set(chars))
        print(len(chars), end=" ")
chars = list(set(chars))
print(len(chars), end=" ")

0 112 136 145 154 159 167 179 180 189 195 196 201 205 210 210 213 214 

In [ ]:
for i in range(0,2000):
    print(chr(i) , i)